In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.analytics.visualizations.charts import CandleChart, Study


from tastytrade.messaging.processors.influxdb import TelegrafHTTPEventProcessor

from tastytrade.analytics.indicators.momentum import hull

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

In [3]:
influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials().url, token=influx_user.token, org=influx_user.org
)
query_api = influxdb.query_api()

In [4]:
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())

In [ ]:
dxlink.router.handler[Channels.Candle].processors

In [ ]:
start_time = datetime(2025, 2, 21, 12, 00)

for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:

    for interval in ["1d", "1h", "30m", "15m", "5m", "1m"]:
        await dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )

In [ ]:
symbols = ["NVDA"]
symbols = ["SPX"]
symbols = ["SPX", "NVDA", "SPY", "QQQ"]
symbols = ["BTC/USD:CXTALP", "SPX", "NVDA", "SPY", "QQQ"]
await dxlink.subscribe(symbols)

In [ ]:
symbol = "BTC/USD:CXTALP{=m}"

dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}"]

In [18]:
from tastytrade.providers.market import MarketDataProvider

stream = MarketDataProvider(dxlink, influxdb)

In [ ]:
stream.download(
    symbol=symbol,
    event_type="CandleEvent",
    start=datetime(2025, 2, 18),
    # debug_mode=True,
)

await stream.subscribe(symbol)

In [ ]:
stream[symbol].sort("time", descending=True).head(5)

In [ ]:
dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}"].sort(
    "time", descending=True
).head(5)


In [ ]:
hull(input_df=stream["SPX{=5m}"], symbol="SPX{=5m}").head(3)

In [12]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,  # The hull function
    params={"length": 20},  # Parameters for hull function
    plot_params={
        "colors": {"Up": "#01FFFF", "Down": "#FF66FE"},  # Green for uptrend  # Red for downtrend
        "width": 2,
    },
    value_column="HMA",
    color_column="HMA_color",
)

In [ ]:
chart1 = CandleChart(
    stream,
    "SPX{=5m}",
    start_time=pd.Timestamp("2025-02-18 14:00").tz_localize("UTC"),
    end_time=pd.Timestamp("2025-02-18 21:00").tz_localize("UTC"),
)
chart1.add_study(hma_study)
chart1.start()

In [15]:
stream.download(
    symbol="SPX{=5m}",
    event_type="CandleEvent",
    start=datetime(2025, 2, 18),
)

In [14]:
await chart1.stop()

In [ ]:
await dxlink.close()

In [24]:
# TODOS

# [x] Widen the plot
# Remove the scroller at the bottom
# move the legent and remove Price (that is obvious)

# Add MACD
# Add RSI
# Add Volume Profile (?? ... /ES, SPY, etc)
# Add velocity metric

# Do not plot study data points passed available data

# [x] HULL - Align the first tick against to the subsequent tick

# ERROR if no study data found